In [ ]:
import numpy as np
import mpmath 
import matplotlib.pyplot as plt
from IPython.display import Audio
import math as mth
from time import time
from contextlib import contextmanager
from scipy import linalg, signal, constants, optimize

In [ ]:
data = np.loadtxt('rho_2dots_dt5e-5_fix.dat')
rho_00_dot1 = data[:,0]
rho_01_dot1 = data[:,1] + 1j*data[:,2]

rho_00_dot2 = data[:,4]
rho_01_dot2 = data[:,5] + 1j*data[:,6]

In [ ]:
data = np.loadtxt('rho_2dots_dt5e-5_fix_nint.dat')
rho_00_dot1_nint = data[:,0]
rho_01_dot1_nint = data[:,1] + 1j*data[:,2]

rho_00_dot2_nint = data[:,4]
rho_01_dot2_nint = data[:,5] + 1j*data[:,6]

In [ ]:
data = np.loadtxt('chi_2dots_fix.dat')
chi_rad_elliot = data[:,0]

In [ ]:
@contextmanager
def timer(prefix=None):
    t_start = time()
    yield
    t_end = time()
    t = t_end - t_start
    
    minutes = np.floor(t / 60)
    seconds = t % 60
    if prefix is None:
        print(minutes,'minutes',seconds,'seconds')
        
    else:
        print(prefix+':',minutes,'minutes',seconds,'seconds')
        
    return Audio('Ring.wav',autoplay=True)

In [ ]:
def get_uvw(rho_00,rho_01):
    u = 2*rho_01.real
    v = 2*rho_01.imag
    w = 1-2*rho_00
    return u,v,w

In [ ]:
u1_elliot,v1_elliot,w1_elliot = get_uvw(rho_00_dot1,rho_01_dot1)
u2_elliot,v2_elliot,w2_elliot = get_uvw(rho_00_dot2,rho_01_dot2)

u1_elliot_nint,v1_elliot_nint,w1_elliot_nint = get_uvw(rho_00_dot1_nint,rho_01_dot1_nint)
u2_elliot_nint,v2_elliot_nint,w2_elliot_nint = get_uvw(rho_00_dot2_nint,rho_01_dot2_nint)

t_elliot = np.linspace(0,10,u1_elliot.size)

In [ ]:
def compare_frequency_spectra_solutions(signal_t_1,signal_t_2,t,minimum_f=-t_elliot.size/2,maximum_f=t_elliot.size/2,sol=' Guess'):
    #plots the freq spec for a given signal.  minimum_f and maximum_f must be in Hz. signal is the variable name
    stop_time = t[-1]
    dt = stop_time/t.size
    
    Signal_w_1 = np.fft.fftshift(np.fft.fft(signal_t_1))
    Signal_w_2 = np.fft.fftshift(np.fft.fft(signal_t_2))
    freq = np.fft.fftshift(np.fft.fftfreq(signal_t_1.size,dt))
    
    min_nfreq = np.argmin(abs(minimum_f - freq))
    max_nfreq = np.argmin(abs(maximum_f - freq))
    
#     plt.figure()
#     plt.plot(t_elliot,phase_error,label='Phase error')
#     plt.legend()
   
    plt.figure()
    plt.semilogy(freq[min_nfreq:max_nfreq],abs(Signal_w_1)[min_nfreq:max_nfreq],label='Chad'+sol)

    plt.semilogy(freq[min_nfreq:max_nfreq],abs(Signal_w_2)[min_nfreq:max_nfreq],label='Elliot Solution')
    plt.legend()
    plt.grid()
    plt.xlabel('Linear Frequency')
    plt.ylabel('|Magnitude|')

    plt.figure()
    
    plt.plot(freq[min_nfreq:max_nfreq],np.angle(Signal_w_1,deg=True)[min_nfreq:max_nfreq],label='Chad'+sol)
    plt.plot(freq[min_nfreq:max_nfreq],np.angle(Signal_w_2,deg=True)[min_nfreq:max_nfreq],label='Elliots Solution')
    plt.ylabel('Phase (degrees)')
    plt.xlabel('Linear Frequency')
    plt.legend()
    
    plt.figure()
    plt.scatter(omega_k/(2*np.pi),np.zeros_like(omega_k))
    plt.xlim(minimum_f,maximum_f)
    plt.show()

In [ ]:
def compare_frequency_spectra_solutions(signal_t_1,signal_t_2,t,minimum_f=None,maximum_f=None,sol=' Guess'):
    #plots the freq spec for a given signal.  minimum_f and maximum_f must be in Hz. signal is the variable name
    stop_time = t[-1]
    dt = stop_time/t.size
    
    Signal_w_1 = np.fft.fftshift(np.fft.fft(signal_t_1))
    Signal_w_2 = np.fft.fftshift(np.fft.fft(signal_t_2))
    freq = np.fft.fftshift(np.fft.fftfreq(signal_t_1.size,dt))

   
    plt.figure()
    plt.semilogy(freq,abs(Signal_w_1),label='Chad'+sol)
    plt.semilogy(freq,abs(Signal_w_2),label='Elliot Solution')
    for frequency in omega_k/(2*np.pi):
        plt.axvline(frequency,c='k')
    plt.legend()
    plt.grid()
    plt.xlabel('Linear Frequency')
    plt.xlim(minimum_f,maximum_f)
    plt.ylabel('|Magnitude|')

    plt.figure()
    plt.plot(freq,np.angle(Signal_w_1,deg=True),label='Chad'+sol)
    plt.plot(freq,np.angle(Signal_w_2,deg=True),label='Elliots Solution')
    for frequency in omega_k/(2*np.pi):
        plt.axvline(frequency,c='k')
    plt.ylabel('Phase (degrees)')
    plt.xlabel('Linear Frequency')
    plt.xlim(minimum_f,maximum_f)
    plt.legend()

In [ ]:
#Define Constants

#General Parameters
omega_0 = 2278.9013 # in rad/picosecond
period = 2*np.pi/omega_0
f0 = omega_0 / (2*np.pi)

stop_time = 10 #in picoseconds

#Dot parameters
sigma = 2278.9013
t0 = 0.0
hbar = 0.65821193
T1 = 1e6
T2 = 2e6
x = np.array([0.0,0.0])
y = np.array([-0.00175,0.00175])

R = y[1] - y[0]
#Misc Parameters
x_hat = np.array([1,0,0])
y_hat = np.array([0,1,0])
r_hat_dot1 = np.array([0,1,0])
r_hat_dot2 = np.array([0,-1,0])

c0 = 299.792458 #um/ps
mu0 = 2.0133545e-04
eps0 = 1/(mu0*c0**2)
print(eps0)
d = 5.2917721e-4
d_vector = d*x_hat
E0 = 1558.8929

#omega_r = d*E0/hbar #+ E_rad*5.2917721e-4/hbar 
f_r = 2.2669
omega_r = 2*np.pi*f_r

In [ ]:
delta_f = 0.5
delta_omega = 2*np.pi*delta_f
B = 4
C = 4
i = np.arange(B+1)
j = np.arange(-B,B+1)
k = np.arange(-C,C+1)

i,j,k = np.meshgrid(i,j,k,indexing='ij')
ij_sum_abs = abs(i) + abs(j)
ij = ij_sum_abs<=B
i = i[ij]
j = j[ij]
k = k[ij]

omega_k = i * omega_0 + j * omega_r + k * delta_omega
omega_k = np.unique(omega_k)

#Remove negative frequencies for real signals 
omega_k = omega_k[omega_k>=0]
omega_k = np.sort(omega_k)

num_frequencies = omega_k.size
num_coefficients = 2*num_frequencies - 1

print('Solving',num_frequencies,'frequencies')
print('Solving',3*num_coefficients,'coefficients')

In [ ]:
def get_gamma(t,omega=omega_k):
    #Creates a matrix of trigonometric basis functions
    return np.hstack((np.cos(omega*t[:,np.newaxis]),np.sin(omega[1:]*t[:,np.newaxis])))

In [ ]:
def get_gamma_t0(t0):
    #Creates a matrix of trigonometric basis functions evaluated at an instant in time, t0
    #Used for enforcing initial conditions in the Newton Solver
    return np.hstack((np.cos(omega_k*t0),np.sin(omega_k[1:]*t0)))

In [ ]:
def get_gamma_prime(t,omega=omega_k):
    #Creates a matrix of derivatives of trig basis functions
    return np.hstack((-omega*np.sin(omega*t[:,np.newaxis]),omega[1:]*np.cos(omega[1:]*t[:,np.newaxis])))

In [ ]:
def get_gamma_double_prime(t,omega=omega_k):
    #Second derivative matrix
    return np.hstack((-omega**2*np.cos(omega*t[:,np.newaxis]),-omega[1:]**2*np.sin(omega[1:]*t[:,np.newaxis])))

In [ ]:
def get_rhos(u,v,w):
    rho_00 = 0.5 - 0.5*w
    rho_01 = 0.5*u + 0.5j*v
    return rho_00, rho_01

In [ ]:
def get_P(u_coeff,v_coeff,w_coeff,t,omega=omega_k):
    #Evaluates the polarization
    delay = t - (R/c0)
    gamma_delay = get_gamma(delay,omega)
    u_delay = gamma_delay@u_coeff
    v_delay = gamma_delay@v_coeff
    w_delay = gamma_delay@w_coeff
    
    rho_00_delay,rho_01_delay = get_rhos(u_delay,v_delay,w_delay)
    
    P = 2*(rho_01_delay.real)*d_vector[:,np.newaxis]
    
    return P

In [ ]:
def get_P_prime(u_coeff,v_coeff,w_coeff,t,omega=omega_k):
    delay = t - (R/c0)
    gamma_prime_delay = get_gamma_prime(delay,omega)
    u_prime_delay = gamma_prime_delay@u_coeff
    v_prime_delay = gamma_prime_delay@v_coeff
    w_prime_delay = gamma_prime_delay@w_coeff
    
    rho_00_prime_delay,rho_01_prime_delay = get_rhos(u_prime_delay,v_prime_delay,w_prime_delay)
    P_prime = 2*(rho_01_prime_delay.real)*d_vector[:,np.newaxis]
    
    return P_prime

In [ ]:
def get_P_doubleprime(u_coeff,v_coeff,w_coeff,t,omega=omega_k):
    delay = t - (R/c0)
    gamma_doubleprime = get_gamma_double_prime(delay,omega)
    u_doubleprime_delay = gamma_doubleprime@u_coeff
    v_doubleprime_delay = gamma_doubleprime@v_coeff
    w_doubleprime_delay = gamma_doubleprime@w_coeff
    
    rho_00_doubleprime_delay,rho_01_doubleprime_delay = get_rhos(u_doubleprime_delay,v_doubleprime_delay,w_doubleprime_delay)
    P_doubleprime = 2*(rho_01_doubleprime_delay.real)*d_vector[:,np.newaxis]
    
    return P_doubleprime

In [ ]:
def get_E_rad(u_coeff,v_coeff,w_coeff,t,r_hat):
    #Evaluates the radiation field from one dot onto another.  In the 2-dot case R is a constant for both dots.
    P = get_P(u_coeff,v_coeff,w_coeff,t)
    P_prime = get_P_prime(u_coeff,v_coeff,w_coeff,t)
    P_doubleprime = get_P_doubleprime(u_coeff,v_coeff,w_coeff,t)
    
    I = np.eye(3)
    
    E_rad = np.dot(I - np.outer(r_hat,r_hat),P_doubleprime/(c0**2*R))
    
    E_rad += np.dot(I-3*np.outer(r_hat,r_hat),P_prime/(c0*R**2))

    E_rad += np.dot(I-3*np.outer(r_hat,r_hat),P/(R**3))
    
    E_rad *= -1/(4*np.pi*eps0)
    
    return E_rad

In [ ]:
def get_chi(t, Er):
    #Takes in a radiation field from a specific dot and returns chi for that dot.
    wave_vector = x_hat
    r = x_hat*0.0
    args = np.dot(wave_vector,r) - omega_0*(t-t0)
    
    #pulse = np.exp(omega_0*(t - t0)**2/(2*sigma**2))
    #pulse = 0
    pulse = 1
    El = (E0*np.cos(args)*pulse)*x_hat[:,np.newaxis]
    
#     plt.figure()
#     plt.plot(t,Er[0,:],label='Er')
#     plt.legend()
    
#     plt.figure()
#     plt.plot(t,El[0,:],label='E laser')
#     plt.legend()
#     plt.show()
    
    E = El + Er
    chi = np.dot(d_vector,E) / hbar
    
    return chi

In [ ]:
t_k = np.linspace(0,stop_time,num_coefficients - 1)

In [ ]:
def plot_error(t,f1,f2,f3,f4,f5,f6):
    plt.figure()
    plt.semilogy(t,abs(f1),label='f1')
    plt.semilogy(t,abs(f2),label='f2')
    plt.semilogy(t,abs(f3),label='f3')
    plt.legend()
    plt.show()

In [ ]:
def plot_chi(t,chi):
    plt.figure()
    plt.plot(t,chi,label='chi')
    plt.legend()
    plt.show()

In [ ]:
def get_f(coefficients,t=t_k):
    #Takes in an array of Fourier coefficients for u,v,w for both dots and returns the error functions in time
    u1_coeff = coefficients[:num_coefficients]
    v1_coeff = coefficients[num_coefficients:2*num_coefficients]
    w1_coeff = coefficients[2*num_coefficients:3*num_coefficients]
    
    u2_coeff = coefficients[3*num_coefficients:4*num_coefficients]
    v2_coeff = coefficients[4*num_coefficients:5*num_coefficients]
    w2_coeff = coefficients[5*num_coefficients:]
    
    gamma_t = get_gamma(t)
    gamma_prime = get_gamma_prime(t)
    gamma_0 = get_gamma_t0(0)
    
    u1 = gamma_t@u1_coeff
    v1 = gamma_t@v1_coeff
    w1 = gamma_t@w1_coeff
    
    u1_prime = gamma_prime@u1_coeff
    v1_prime = gamma_prime@v1_coeff
    w1_prime = gamma_prime@w1_coeff
    
    u2 = gamma_t@u2_coeff
    v2 = gamma_t@v2_coeff
    w2 = gamma_t@w2_coeff
    
    u2_prime = gamma_prime@u2_coeff
    v2_prime = gamma_prime@v2_coeff
    w2_prime = gamma_prime@w2_coeff
     
    Er_dot1 = get_E_rad(u1_coeff,v1_coeff,w1_coeff,t,r_hat_dot1)
    Er_dot2 = get_E_rad(u2_coeff,v2_coeff,w2_coeff,t,r_hat_dot2)
    
    chi1 = get_chi(t,Er_dot2)
    chi2 = get_chi(t,Er_dot1)
    
    f1_dot1 = -omega_0*v1 - u1_prime
    f2_dot1 = omega_0*u1 - 2*chi1*w1 - v1_prime
    f3_dot1 = 2*chi1*v1 - w1_prime
    f4_dot1 = gamma_0@u1_coeff
    f5_dot1 = gamma_0@v1_coeff
    f6_dot1 = gamma_0@w1_coeff + 1
    
    f1_dot2 = -omega_0*v2 - u2_prime
    f2_dot2 = omega_0*u2 - 2*chi2*w2 - v2_prime
    f3_dot2 = 2*chi2*v2 - w2_prime
    f4_dot2 = gamma_0@u2_coeff
    f5_dot2 = gamma_0@v2_coeff
    f6_dot2 = gamma_0@w2_coeff + 1
    
    fk_dot1 = np.hstack((f1_dot1,f2_dot1,f3_dot1,f4_dot1,f5_dot1,f6_dot1))
    fk_dot2 = np.hstack((f1_dot2,f2_dot2,f3_dot2,f4_dot2,f5_dot2,f6_dot2))
    
    return np.hstack((fk_dot1,fk_dot2))

In [ ]:
def get_chirad(coefficients,t=t_k):
    #Takes in an array of Fourier coefficients for u,v,w for both dots and returns the error functions in time
    u1_coeff = coefficients[:num_coefficients]
    v1_coeff = coefficients[num_coefficients:2*num_coefficients]
    w1_coeff = coefficients[2*num_coefficients:3*num_coefficients]
    
    u2_coeff = coefficients[3*num_coefficients:4*num_coefficients]
    v2_coeff = coefficients[4*num_coefficients:5*num_coefficients]
    w2_coeff = coefficients[5*num_coefficients:]
    
    gamma_t = get_gamma(t)
    gamma_prime = get_gamma_prime(t)
    gamma_0 = get_gamma_t0(0)
    
    u1 = gamma_t@u1_coeff
    v1 = gamma_t@v1_coeff
    w1 = gamma_t@w1_coeff
    
    u1_prime = gamma_prime@u1_coeff
    v1_prime = gamma_prime@v1_coeff
    w1_prime = gamma_prime@w1_coeff
    
    u2 = gamma_t@u2_coeff
    v2 = gamma_t@v2_coeff
    w2 = gamma_t@w2_coeff
    
    u2_prime = gamma_prime@u2_coeff
    v2_prime = gamma_prime@v2_coeff
    w2_prime = gamma_prime@w2_coeff
     
    Er_dot1 = get_E_rad(u1_coeff,v1_coeff,w1_coeff,t,r_hat_dot1)
    Er_dot2 = get_E_rad(u2_coeff,v2_coeff,w2_coeff,t,r_hat_dot2)
    
    chi_rad = np.dot(d_vector,Er_dot1) / hbar
    chi = get_chi(t,Er_dot2)
    
    return chi_rad,chi

In [ ]:
#Creates initial guesses for u,v,w for both dots using least square approximation
gamma = get_gamma(t_elliot)
W = gamma.T@gamma
W_inv = linalg.inv(W)

u1_guess = W_inv@(gamma.T@u1_elliot) 
v1_guess = W_inv@(gamma.T@v1_elliot)
w1_guess = W_inv@(gamma.T@w1_elliot)

u2_guess = W_inv@(gamma.T@u2_elliot)
v2_guess = W_inv@(gamma.T@v2_elliot)
w2_guess = W_inv@(gamma.T@w1_elliot)

coefficients_guess = np.hstack((u1_guess,v1_guess,w1_guess,u2_guess,v2_guess,w2_guess)) 

In [ ]:
u1_guess = gamma@u1_guess
v1_guess = gamma@v1_guess
w1_guess = gamma@w1_guess

u2_guess = gamma@u2_guess
v2_guess = gamma@v2_guess
w2_guess = gamma@w2_guess

In [ ]:
compare_frequency_spectra_solutions(u1_guess,u1_elliot,t_elliot,360,364)

In [ ]:
#Solve system using Scipy Optimize Package
with timer():
    coeff = optimize.fsolve(get_f,coefficients_guess)

In [ ]:
Audio('Ring.wav',autoplay=True)

In [ ]:
u1_coeff = coeff[:num_coefficients]
v1_coeff = coeff[num_coefficients:2*num_coefficients]
w1_coeff = coeff[2*num_coefficients:3*num_coefficients]

u2_coeff = coeff[3*num_coefficients:4*num_coefficients]
v2_coeff = coeff[4*num_coefficients:5*num_coefficients]
w2_coeff = coeff[5*num_coefficients:]

u1 = gamma@u1_coeff
v1 = gamma@v1_coeff
w1 = gamma@w1_coeff

u2 = gamma@u2_coeff
v2 = gamma@v2_coeff
w2 = gamma@w2_coeff

In [ ]:
plt.figure()
plt.plot(t_elliot,u1_guess,label='u1 Guess')
plt.plot(t_elliot,u1_elliot,label='u1  Solution Elliot')
plt.xlim(0,0.05)
plt.legend()
plt.show()

In [ ]:
compare_frequency_spectra_solutions(u1,u1_elliot,t_elliot,362,363)

In [ ]:
plt.plot((np.fft.fftshift((np.angle(np.fft.fft(u1),deg=True)-np.angle(np.fft.fft(u1_elliot),deg=True)+360)%360)))

# plt.xlim(0,1000)

In [ ]:
print(f_r/2,9/8)

In [ ]:
plt.plot(np.angle(u1),label='chad')
plt.plot(np.angle(u1_elliot),label='elliot')
plt.xlim(0,200)

In [ ]:
compare_frequency_spectra_solutions(w1,w1_elliot,t_elliot,719,726)

In [ ]:
plt.figure()
plt.plot(t_elliot,u1,label='Chad solution u1')
plt.plot(t_elliot,u1_elliot,label='Elliot solution u1')
plt.xlim(0,8/9)
plt.legend()
plt.show()

In [ ]:
compare_frequency_spectra_solutions(u1,u1_elliot,t_elliot,0,2)

In [ ]:
plt.figure()
plt.plot(t_elliot,v1,label='Chad solution v1')
plt.plot(t_elliot,v1_elliot,label='Elliot solution v1')
plt.xlim(0,0.05)
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(t_elliot,w1,label='Chad solution w1')
plt.plot(t_elliot,w1_elliot,label='Elliot solution w1')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(t_elliot,np.sqrt(u1**2 + v1**2),label='chad')
plt.plot(t_elliot,np.sqrt(u1_elliot**2 + v1_elliot**2),label='elliot')
plt.legend()


In [ ]:
plt.figure()
plt.plot(t_elliot,chi_chad,label='chi Chad')
plt.plot(t_elliot,chi_elliot,label='chi Elliot')
plt.legend()
plt.xlim(0,0.1)
plt.show()

In [ ]:
plt.figure()
plt.plot(t_elliot,chi_rad_chad,label='chi radiation Chad')
plt.plot(t_elliot,chi_rad_elliot,label='chi radiation Elliot')
plt.legend()
plt.xlim(0,0.1)
plt.show()